In [86]:
!pip install -U "tensorflow-text==2.15.*"
!pip install -U "tf-models-official==2.15.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 5.3 MB/s eta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=9fa344ed4a0000b0ba7f9f803d3a21240a98d3eab2044d7123ba11d19252f1f1
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
email_data = pd.read_csv('email.csv')
email_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
email_data['Message'][2]

"Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"

In [4]:
email_data.shape

(5573, 2)

In [5]:
email_data['Category'].value_counts()

,count
Category,
ham,4825
spam,747
"{""mode"":""full""",1


In [6]:
747/4825

0.15481865284974095

# This means that the data is imbalanced
  15 % is spam and the rest is not



# We should balance the data
 1st (under sampling the majority class which is ham)

In [7]:
spam_email = email_data[email_data['Category']=='spam']
spam_email.head(5)

,Category,Message
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [8]:
spam_email.shape

(747, 2)

In [9]:
not_spam_email = email_data[email_data['Category']=='ham']
not_spam_email.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
6,ham,Even my brother is not like to speak with me. ...


# under sampling majority class (ham)

In [10]:
not_spam_email_under_sample = not_spam_email.sample(spam_email.shape[0])
not_spam_email_under_sample.shape

(747, 2)

In [11]:
email_data_under_sample = pd.concat([not_spam_email_under_sample,spam_email])
email_data_under_sample.sample(4)

,Category,Message
4474,ham,S but not able to sleep.
2256,ham,"I just lov this line: ""Hurt me with the truth,..."
2711,spam,Hope you enjoyed your new content. text stop t...
2243,ham,Nope wif my sis lor... Aft bathing my dog then...


In [12]:
email_data_under_sample.shape

(1494, 2)

In [13]:
email_data_under_sample.Category.value_counts()

,count
Category,
ham,747
spam,747


In [14]:
email_data_under_sample['Spam'] = email_data_under_sample['Category'].apply(lambda x: 1 if x=='spam' else 0)
email_data_under_sample.sample(4)

,Category,Message,Spam
4164,ham,I told that am coming on wednesday.,0
5515,ham,You are a great role model. You are giving so ...,0
3413,ham,No she didnt. I will search online and let you...,0
474,spam,Want 2 get laid tonight? Want real Dogging loc...,1


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test = train_test_split(email_data_under_sample['Message'],email_data_under_sample['Spam'],test_size=0.2)

# Now lets import BERT model and get embeding vectors for few sample statements


In [17]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [18]:
def get_sentence(sentences):
  Preprocessed_text = bert_preprocess(sentences)
  encoded_text = bert_encoder(Preprocessed_text)
  return encoded_text['pooled_output']

In [19]:
get_sentence([
    "500$ discount. hurry up",
    "Bhavin, are you up for a volleybal game tomorrow?",
    "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"
    ])

<tf.Tensor: shape=(3, 768), dtype=float32, numpy=
array([[-0.8435168 , -0.5132724 , -0.8884571 , ..., -0.74748844,
        -0.75314724,  0.91964483],
       [-0.87208354, -0.50543964, -0.94446665, ..., -0.85847497,
        -0.7174534 ,  0.88082975],
       [-0.70380396, -0.5027498 , -0.96400714, ..., -0.91099584,
        -0.5792529 ,  0.6098899 ]], dtype=float32)>

In [20]:
test = get_sentence([
    "banana",
    "grapes",
    "mango",
    "Ronaldo",
    "elon musk",
    "bill gates",
    "Messi",
    "Muller"
]
)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([test[0]],[test[1]])

array([[0.9911088]], dtype=float32)

In [83]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([test[0]],[test[6]])

array([[0.76199055]], dtype=float32)

# Building the model (functional model)

In [23]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='Inputs')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network
layer_1 = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
layer_2 = tf.keras.layers.Dense(1, activation='sigmoid')(layer_1)

# Model
model = tf.keras.Model(inputs=text_input, outputs=layer_2)



In [24]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Inputs (InputLayer)         [(None,)]                    0         []                            
                                                                                                  
 keras_layer (KerasLayer)    {'input_type_ids': (None,    0         ['Inputs[0][0]']              
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [25]:
len(x_train)

1195

In [26]:
len(x_test)

299

In [27]:
Metrics = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=Metrics)

In [28]:
model.fit(x_train,y_train,epochs=10)

Epoch 1/10
38/38 [==============================] - 626s 16s/step - loss: 0.6575 - accuracy: 0.6050 - precision: 0.6061 - recall: 0.5712
Epoch 2/10
38/38 [==============================] - 610s 16s/step - loss: 0.5201 - accuracy: 0.8017 - precision: 0.7797 - recall: 0.8339
Epoch 3/10
38/38 [==============================] - 607s 16s/step - loss: 0.4715 - accuracy: 0.8234 - precision: 0.8185 - recall: 0.8254
Epoch 4/10
38/38 [==============================] - 610s 16s/step - loss: 0.4115 - accuracy: 0.8561 - precision: 0.8266 - recall: 0.8966
Epoch 5/10
38/38 [==============================] - 606s 16s/step - loss: 0.3804 - accuracy: 0.8711 - precision: 0.8633 - recall: 0.8780
Epoch 6/10
38/38 [==============================] - 607s 16s/step - loss: 0.3620 - accuracy: 0.8762 - precision: 0.8519 - recall: 0.9068
Epoch 7/10
38/38 [==============================] - 605s 16s/step - loss: 0.3413 - accuracy: 0.8845 - precision: 0.8681 - recall: 0.9034
Epoch 8/10
38/38 [=======================

In [29]:
model.evaluate(x_test, y_test)


10/10 [==============================] - 153s 15s/step - loss: 0.3166 - accuracy: 0.8963 - precision: 0.9565 - recall: 0.8408


[0.3166045546531677, 0.8963210582733154, 0.95652174949646, 0.8407643437385559]

In [30]:
import numpy as np

y_predicted = model.predict(x_test)
y_predicted = y_predicted.flatten()

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

10/10 [==============================] - 157s 16s/step


array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1])

In [32]:
y_test.sample(5)

,Spam
923,0
1118,1
3001,0
672,1
722,0


In [31]:
reviews = [
    'Enter a chance to win $5000, hurry up, offer valid until march 31, 2021',
    'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr warranty. 150ppm. 16 . p pÂ£3.99',
    'it to 80488. Your 500 free text messages are valid until 31 December 2005.',
    'Hey Sam, Are you coming for a cricket game tomorrow',
    "Why don't you wait 'til at least wednesday to see if you get your ."
]
prediction = model.predict(reviews)
prediction = prediction.flatten()
prediction = np.where(prediction > 0.5, 'Spam','Not spam')
prediction

1/1 [==============================] - 4s 4s/step


array(['Spam', 'Spam', 'Spam', 'Not spam', 'Not spam'], dtype='<U8')